## Pull ZIP Code data

For each ZIP code (**NOT** ZCTA) in Rhode Island, provide an approximation of:
  * The total vote in 2020
  * The Democratic and Republican vote for President in 2020
  * The total population
  * The black and Hispanic population

Computes based on:
  * Rhode Island precinct-level vote totals (and shapefiles)
  * ACS 5-year population totals
  * HUD USPS TRACT/ZIP crosswalks

In [1]:
import json
import os
import re
from collections import Counter
from pathlib import Path
from typing import Union
from urllib import parse

import geopandas as gpd
import pandas as pd
import requests
import us
from bs4 import BeautifulSoup
from census import Census
from dotenv import load_dotenv
from tqdm.notebook import tqdm

load_dotenv()

True

In [2]:
DATA_DIR = Path.cwd() / ".." / ".." / ".." / "data" / "demographics"

DATA_DIR.mkdir(exist_ok=True, parents=True)

TRACT_ZIP_FILE = DATA_DIR / "TRACT_ZIP_032021.xlsx"

# Redownload precinct data if True
REDOWNLOAD = False

In [3]:
def get_soup(url: str) -> BeautifulSoup:
    """Download a URL into BeautifulSoup"""
    response = requests.get(url)
    return BeautifulSoup(response.content, "html.parser")


def get_last(url: str) -> str:
    """Get the last folder from a URL"""
    return url.rstrip("/").split("/")[-1]


def download_file(url: str, outpath: Union[str, Path], chunk_size: int = 8192):
    """Download file at `url` to `outpath`"""
    with requests.get(url, stream=True) as response:
        length = int(response.headers.get("Content-length", 0))
        with open(outpath, "wb") as outfile:
            for chunk in tqdm(
                response.iter_content(chunk_size=chunk_size), total=length // chunk_size
            ):
                outfile.write(chunk)

### Download precinct level result data

In [4]:
BASE_URL = "https://www.ri.gov/election/results/2020/general_election/data/"

soup = get_soup(BASE_URL)
elt = soup.find("h3", text="Results by city/town")

city_urls = [
    parse.urljoin(BASE_URL, local_elt.get("href"))
    for local_elt in elt.parent.find_all("a")
]

In [5]:
def parse_tr(tr):
    """Parse a tr into results"""
    vote_count_texts = [
        x.text.replace("\xa0", " ")
        for x in tr.find("ul", class_="mailinbreakout").find_all("li")
    ]
    vote_count_dict = {}
    for text in vote_count_texts:
        place, count = re.match(r"^([A-Za-z ]+): (\d+)$", text).groups()
        count = int(count)
        vote_count_dict[place] = count
    total = sum(vote_count_dict.values())
    candidate_name = tr.find("td", class_="candidate").text.strip()
    return {
        "candidate_name": candidate_name,
        "total": total,
        "vote_count_details": vote_count_dict,
    }

In [6]:
if REDOWNLOAD or not (DATA_DIR / "precinct_results_ri.json").exists():
    by_city = {}
    for city_url in tqdm(city_urls, desc="Overall..."):
        soup = get_soup(city_url)
        precinct_urls = [
            parse.urljoin(city_url, local_elt.get("href"))
            for local_elt in soup.find("div", id="byPrecinct").find_all("a")
        ]
        by_precinct = {}
        city_name = get_last(city_url)

        for precinct_url in tqdm(precinct_urls, desc=get_last(city_url)):
            soup = get_soup(precinct_url)
            soup.find_all("div", class_="raceResults")[0].find("table").get("summary")
            votes_for_candidates_by_race = {}
            for div in soup.find_all("div", class_="raceResults"):
                race = div.find("table").get("summary")
                votes_for_candidates = []
                for tr in div.find("tbody").find_all("tr"):
                    votes_for_candidates.append(parse_tr(tr))
                votes_for_candidates_by_race[race] = votes_for_candidates
            by_precinct[get_last(precinct_url)] = votes_for_candidates_by_race

        by_city[get_last(city_url)] = by_precinct

    with open(DATA_DIR / "precinct_results_ri.json", "wt") as outfile:
        json.dump(by_city, outfile)
else:
    with open(DATA_DIR / "precinct_results_ri.json", "rt") as infile:
        by_city = json.load(infile)

In [7]:
# Make sure each precinct name appears only once
Counter(Counter(k for v in by_city.values() for k in v).values())

Counter({1: 500})

In [8]:
def get_from_list(lst, candidate_name):
    for val in lst:
        if val["candidate_name"].startswith(candidate_name):
            return val["total"]


def get_total(lst):
    return sum(val["total"] for val in lst)

In [9]:
precinct_df = pd.DataFrame.from_records(
    [
        (
            precinct_name,
            get_from_list(
                precinct_data["Presidential Electors For:"], "Joseph R. Biden"
            ),
            get_from_list(
                precinct_data["Presidential Electors For:"], "Donald J. Trump"
            ),
            get_total(precinct_data["Presidential Electors For:"]),
        )
        for city_data in by_city.values()
        for precinct_name, precinct_data in city_data.items()
    ],
    columns=["precinct_name", "dem_total", "rep_total", "total"],
)

### Download precint shapefiles

In [10]:
PRECINCT_SHAPEFILE_ONLINE = (
    "https://github.com/mggg-states/RI-shapefiles/blob/master/RI_precincts.zip?raw=true"
)
PRECINCT_SHAPEFILE_LOCAL = DATA_DIR / "precinct_shapes.zip"

In [11]:
download_file(PRECINCT_SHAPEFILE_ONLINE, PRECINCT_SHAPEFILE_LOCAL)

  0%|          | 0/413 [00:00<?, ?it/s]

In [12]:
gdf = gpd.read_file(f"zip://{PRECINCT_SHAPEFILE_LOCAL}")

In [13]:
precinct_df = (
    gdf[["NAME", "geometry"]]
    .rename(columns={"NAME": "precinct_name"})
    .merge(precinct_df, on="precinct_name")
)

In [14]:
precinct_df

,precinct_name,geometry,dem_total,rep_total,total
0,3206,"POLYGON ((-71.59728 41.52114, -71.59425 41.520...",941,458,1443
1,2102,"MULTIPOLYGON (((-71.32694 41.51443, -71.32685 ...",1429,451,1916
2,2608,"POLYGON ((-71.39121 41.85671, -71.39236 41.857...",1195,256,1482
3,2823,"POLYGON ((-71.42801 41.80906, -71.42799 41.808...",915,247,1175
4,2855,"POLYGON ((-71.44787 41.83140, -71.44719 41.831...",1118,363,1503
...,...,...,...,...,...
413,2701,"MULTIPOLYGON (((-71.29807 41.58077, -71.29804 ...",72,43,119
414,2810,"POLYGON ((-71.40536 41.82267, -71.40436 41.821...",98,12,111
415,2836,"POLYGON ((-71.41189 41.83637, -71.41188 41.836...",11,1,12
416,2807,"POLYGON ((-71.40906 41.82750, -71.40955 41.827...",4,0,4


### Download ZCTA files

In [15]:
TRACT_SHAPEFILE_LOCAL = DATA_DIR / "tract_shapes.zip"
download_file(us.states.RI.shapefile_urls("tract"), TRACT_SHAPEFILE_LOCAL)

ZCTA_SHAPEFILE_LOCAL = DATA_DIR / "zcta_shapes.zip"
download_file(us.states.RI.shapefile_urls("zcta"), ZCTA_SHAPEFILE_LOCAL)

  0%|          | 0/86 [00:00<?, ?it/s]

  0%|          | 0/90 [00:00<?, ?it/s]

In [16]:
tract_gdf = gpd.read_file(f"zip://{TRACT_SHAPEFILE_LOCAL.resolve()}")
zcta_gdf = gpd.read_file(f"zip://{ZCTA_SHAPEFILE_LOCAL.resolve()}")

In [17]:
# Fix CRS to RI state plane (ft)
tract_gdf = tract_gdf.to_crs(epsg=3438)
zcta_gdf = zcta_gdf.to_crs(epsg=3438)
precinct_df = precinct_df.to_crs(epsg=3438)

In [18]:
merged_tract_precinct = gpd.overlay(
    tract_gdf[["GEOID10", "geometry"]].rename(columns={"GEOID10": "geoid10"}),
    precinct_df,
    how="intersection",
)

merged_zcta_precinct = gpd.overlay(
    zcta_gdf[["GEOID10", "geometry"]].rename(columns={"GEOID10": "geoid10"}),
    precinct_df,
    how="intersection",
)

In [19]:
merged_tract_precinct["geo_area"] = merged_tract_precinct.area

merged_tract_precinct["total_precinct_area"] = merged_tract_precinct.groupby(
    "precinct_name"
)["geo_area"].transform("sum")

merged_tract_precinct["prop_precinct_area"] = (
    merged_tract_precinct["geo_area"] / merged_tract_precinct["total_precinct_area"]
)

# Assign precinct voting data proportionally to _tract_
merged_tract_precinct["dem_local"] = (
    merged_tract_precinct["dem_total"] * merged_tract_precinct["prop_precinct_area"]
)
merged_tract_precinct["rep_local"] = (
    merged_tract_precinct["rep_total"] * merged_tract_precinct["prop_precinct_area"]
)
merged_tract_precinct["local"] = (
    merged_tract_precinct["total"] * merged_tract_precinct["prop_precinct_area"]
)

In [20]:
merged_zcta_precinct["geo_area"] = merged_zcta_precinct.area

merged_zcta_precinct["total_precinct_area"] = merged_zcta_precinct.groupby(
    "precinct_name"
)["geo_area"].transform("sum")

merged_zcta_precinct["prop_precinct_area"] = (
    merged_zcta_precinct["geo_area"] / merged_zcta_precinct["total_precinct_area"]
)

# Assign precinct voting data proportionally to _zcta_
merged_zcta_precinct["dem_local"] = (
    merged_zcta_precinct["dem_total"] * merged_zcta_precinct["prop_precinct_area"]
)
merged_zcta_precinct["rep_local"] = (
    merged_zcta_precinct["rep_total"] * merged_zcta_precinct["prop_precinct_area"]
)
merged_zcta_precinct["local"] = (
    merged_zcta_precinct["total"] * merged_zcta_precinct["prop_precinct_area"]
)

In [21]:
# Make sure assignments are correct
assert (
    merged_tract_precinct.groupby("precinct_name")["local"]
    .sum()
    .round()
    .sort_index()
    .values
    == merged_tract_precinct.drop_duplicates("precinct_name")
    .sort_values(by="precinct_name")["total"]
    .values
).all()

assert (
    merged_zcta_precinct.groupby("precinct_name")["local"]
    .sum()
    .round()
    .sort_index()
    .values
    == merged_zcta_precinct.drop_duplicates("precinct_name")
    .sort_values(by="precinct_name")["total"]
    .values
).all()

### Open TRACT/ZIP File

In [22]:
# You can download this file at https://www.huduser.gov/portal/datasets/usps_crosswalk.html
# Look for TRACT-ZIP (This was 1st Quarter 2021)

tract_zip_df = pd.read_excel(TRACT_ZIP_FILE)
tract_zip_df["zipcode"] = tract_zip_df["ZIP"].apply(lambda x: f"{x:05d}")
tract_zip_df["geoid10"] = tract_zip_df["TRACT"].apply(lambda x: f"{x:011d}")

In [23]:
# Assign each vote total to the tract based on the residential ratio
hold = tract_zip_df[["RES_RATIO", "zipcode", "geoid10"]].merge(
    merged_tract_precinct[["dem_local", "rep_local", "local", "geoid10"]], on="geoid10"
)

hold["dem_local"] = hold["dem_local"] * hold["RES_RATIO"]
hold["rep_local"] = hold["rep_local"] * hold["RES_RATIO"]
hold["local"] = hold["local"] * hold["RES_RATIO"]

final_df = hold.groupby("zipcode")[["dem_local", "rep_local", "local"]].sum()

### Pull Census data for zipcode based file

In [24]:
c = Census(os.environ.get("CENSUS_API_KEY"))

In [25]:
census_df = pd.DataFrame(
    c.acs5.state_county_tract(
        [
            "B01001_001E",  # Total population
            "B01001B_001E",  # Black population
            "B01001I_001E",  # Hispanic population
            "B02009_001E",  # Any black population
        ],
        "44",
        "*",
        "*",
    )
)

In [26]:
census_df["geoid10"] = census_df["state"] + census_df["county"] + census_df["tract"]
census_df = census_df.drop(columns=["state", "county", "tract"])

In [27]:
census_df.rename(
    columns={
        "B01001_001E": "total_population",
        "B01001B_001E": "black_population",
        "B01001I_001E": "hispanic_population",
        "B02009_001E": "any_black_population",
    },
    inplace=True,
)

In [28]:
census_df

,total_population,black_population,hispanic_population,any_black_population,geoid10
0,4932.0,0.0,63.0,15.0,44007013201
1,5176.0,890.0,2024.0,1048.0,44007000101
2,7773.0,425.0,6116.0,532.0,44007001600
3,4979.0,941.0,2990.0,1176.0,44007002200
4,3076.0,370.0,1107.0,533.0,44007002500
...,...,...,...,...,...
239,5084.0,13.0,135.0,13.0,44007013102
240,6394.0,391.0,599.0,705.0,44007010400
241,5111.0,457.0,105.0,590.0,44007010502
242,2376.0,8.0,42.0,52.0,44007010701


In [29]:
hold = tract_zip_df[["RES_RATIO", "zipcode", "geoid10"]].merge(
    census_df[
        [
            "total_population",
            "black_population",
            "hispanic_population",
            "any_black_population",
            "geoid10",
        ]
    ],
    on="geoid10",
)

hold["total_population"] = hold["total_population"] * hold["RES_RATIO"]
hold["black_population"] = hold["black_population"] * hold["RES_RATIO"]
hold["hispanic_population"] = hold["hispanic_population"] * hold["RES_RATIO"]

In [30]:
final_df.join(
    hold.groupby("zipcode")[
        [
            "total_population",
            "black_population",
            "hispanic_population",
            "any_black_population",
        ]
    ].sum()
).rename(
    columns={
        "dem_local": "dem_vote",
        "rep_local": "gop_vote",
        "local": "total_vote",
    }
).to_csv(
    DATA_DIR / "combined_demo_data_by_zipcode.csv", index=True
)

### Pull Census data for ZCTA based file

In [31]:
census_df = pd.DataFrame(
    c.acs5.zipcode(
        [
            "B01001_001E",  # Total population
            "B01001B_001E",  # Black population
            "B01001I_001E",  # Hispanic population
            "B02009_001E",  # Any black population
        ],
        "*",
    )
)

In [32]:
census_df = census_df.rename(
    columns={
        "B01001_001E": "total_population",
        "B01001B_001E": "black_population",
        "B01001I_001E": "hispanic_population",
        "B02009_001E": "any_black_population",
        "zip code tabulation area": "zcta",
    }
)

In [33]:
merged_zcta_precinct["zcta"] = merged_zcta_precinct["geoid10"].str[2:]

In [34]:
merged_zcta_precinct[["zcta", "dem_local", "rep_local", "local"]].groupby(
    "zcta"
).sum().rename(
    columns={
        "dem_local": "dem_vote",
        "rep_local": "gop_vote",
        "local": "total_vote",
    }
).merge(
    census_df, left_index=True, right_on="zcta"
).to_csv(
    DATA_DIR / "combined_demo_data_by_zcta.csv", index=True
)